In [ ]:
import pandas as pd 
import numpy as np 
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.fasttext import FastText
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('spam_ham_dataset.csv',sep=',',encoding='latin-1')
df.sample(5)

,Unnamed: 0,label,text,label_num
1098,2415,ham,"Subject: hpl noms . for jan . 19 , 2001\r\n( s...",0
4190,5025,spam,Subject: click here to improve your wellbeing ...,1
2462,228,ham,Subject: road - show . com q 4 i . com choose ...,0
2673,2698,ham,Subject: duke energy field 8 / 00\r\nplease ch...,0
2640,4864,spam,Subject: business ideas : march 2005\r\nhows i...,1


In [ ]:
X = df['text']
y = df['label']

In [ ]:
from text import Preprocessing
preprocess = Preprocessing(lemmatize=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
X.shape

(5171,)

In [ ]:
X = preprocess.Normalize(X)
X.sample(5)

265     subject enron hpl actuals for september teco t...
322     subject days and the above referenced days hav...
2116    subject re april spot tickets the spot deals a...
3722    subject enron hpl actuals for june teco tap hp...
1673    subject deal extension for for victor can you ...
Name: text, dtype: object

In [ ]:
X = preprocess.tokenize(X)
X.sample(5)

1084    [subject, midcoast, market, deal, ticket, numb...
4509            [subject, update, april, update, version]
1803    [subject, intrastate, gas, hi, daren, bryan, h...
4853    [subject, doctor, visit, stop, waste, money, p...
3243    [subject, test, dfarmer, enron, com, doug, ple...
Name: text, dtype: object

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

--2022-07-24 16:56:54--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  37.2MB/s    in 18s     

2022-07-24 16:57:13 (35.6 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [ ]:
!unzip /content/wiki-news-300d-1M.vec.zip

Archive:  /content/wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [ ]:
!rm /content/wiki-news-300d-1M.vec.zip

In [20]:
import io
def load_fasttext(file):
    f = io.open(file,'r',encoding = 'utf-8',newline='\n',errors = 'ignore')
    n, d = map(int, f.readline().split())
    print(d)
    data = dict() #{}
    for line in tqdm(f,colour = 'red'):
      tokens = line.strip().split(' ')
      data[tokens[0]] = np.array(list(map(float, tokens[1:])))
    return data

In [17]:
load_fasttext('/content/wiki-news-300d-1M.vec')

300


In [21]:
data = load_fasttext('/content/wiki-news-300d-1M.vec')

300


999994it [01:23, 11949.56it/s]


In [32]:
data['Nandhini']

array([ 7.800e-03, -3.850e-02, -8.810e-02,  2.100e-03, -1.596e-01,
        6.070e-02, -2.620e-02,  3.910e-02,  6.900e-03,  6.310e-02,
        7.570e-02,  1.100e-02,  7.000e-03, -1.643e-01,  5.270e-02,
        1.591e-01,  1.870e-02,  1.575e-01, -3.080e-02, -7.010e-02,
       -1.569e-01, -5.230e-02, -2.080e-02, -5.350e-02, -1.048e-01,
        1.076e-01, -4.330e-02,  7.400e-03, -9.590e-02, -6.000e-02,
        8.330e-02, -1.030e-01,  8.130e-02,  5.530e-02, -2.080e-02,
       -3.870e-02,  1.043e-01,  9.530e-02, -2.420e-02, -1.204e-01,
       -2.840e-02, -5.900e-03, -8.560e-02,  1.800e-03,  4.870e-02,
       -1.300e-03,  2.780e-02, -1.018e-01,  1.046e-01,  1.009e-01,
       -5.040e-02, -7.230e-02, -8.842e-01,  5.700e-02, -3.110e-02,
        1.200e-02, -1.138e-01,  3.720e-02,  4.100e-02,  5.800e-03,
       -6.400e-02,  4.760e-02,  1.690e-02,  7.370e-02, -1.066e-01,
        3.340e-02,  5.400e-03,  1.488e-01,  9.290e-02, -3.870e-02,
       -1.106e-01, -3.670e-02, -1.290e-02, -3.790e-02, -9.040e